#  Linear advection equation

Solves

$$
q_t + u q_x = 0
$$

This file simulates the problem shown in Figure 6.1.

In [ ]:
%config InlineBackend.figure_format = 'svg'
from clawpack import pyclaw
from clawpack import riemann
import numpy as np
import matplotlib.pyplot as plt

This function calls clawpack to solve the problem.

In [ ]:
def solve(ncell=100, order=1, limiter=0, kernel_language='Fortran', 
          solver_type='classic', lim_type=2, weno_order=5, time_integrator='SSP33'):

    if kernel_language == 'Fortran':
        riemann_solver = riemann.advection_1D
    elif kernel_language == 'Python':
        riemann_solver = riemann.advection_1D_py.advection_1D
            
    if solver_type=='classic':
        solver = pyclaw.ClawSolver1D(riemann_solver)
        solver.order = order
    elif solver_type=='sharpclaw':
        solver = pyclaw.SharpClawSolver1D(riemann_solver)
        solver.lim_type = lim_type
        solver.weno_order = weno_order
        solver.time_integrator = time_integrator
        if time_integrator == 'SSPLMMk3':
            solver.lmm_steps = 5
            solver.check_lmm_cond = True
    else: raise Exception('Unrecognized value of solver_type.')

    solver.kernel_language = kernel_language
    solver.limiters = limiter
    solver.all_bcs = pyclaw.BC.periodic
    solver.dt_initial = 1.e99
    solver.cfl_desired = 0.8
    solver.cfl_max = 1.0

    xmin, xmax = 0.0, 1.0
    domain = pyclaw.Domain( (xmin,), (xmax,), (ncell,))
    x = domain.grid.p_centers

    solution = pyclaw.Solution(solver.num_eqn, domain)
    beta, x0 = 200.0, 0.3
    solution.q[0, :] = np.exp(-beta * (x-x0)**2)
    solution.q[0, :] += (np.abs(x - 0.7) < 0.1) * 1.0
    solution.state.problem_data['u'] = 1.0 # advection velocity

    claw = pyclaw.Controller()
    claw.tfinal = 10.0
    claw.keep_copy = True       # Keep solution data in memory for plotting
    claw.output_format = None   # Don't write solution data to file
    claw.num_output_times = 50  # Write 50 output frames
    claw.verbosity = 0
    claw.solver = solver
    claw.solution = solution

    status = claw.run()

    return claw


Plot initial solution and at a later time.

In [ ]:
def make_plots(claw):
    x = claw.grid.x.centers
    q0 = claw.frames[0].q[0, :]

    plt.figure(figsize=(10, 5))

    plt.subplot(121)
    f = 5
    q = claw.frames[f].q[0, :]
    plt.plot(x, q0, '-', label='Exact')
    plt.plot(x, q, 'o', fillstyle='none', label='Numerical')
    plt.axis([0, 1, -0.5, 1.5])
    plt.title('Time = ' + str(claw.frames[f].t))
    plt.legend()

    plt.subplot(122)
    f = 25
    q = claw.frames[f].q[0, :]
    plt.plot(x, q0, '-', label='Exact')
    plt.plot(x, q, 'o', fillstyle='none', label='Numerical')
    plt.axis([0, 1, -0.5, 1.5])
    plt.title('Time = ' + str(claw.frames[f].t))
    plt.legend()

## First order (upwind) scheme

In [ ]:
claw = solve(order=1, limiter=0)
make_plots(claw)

## Lax-Wendroff without limiter

In [ ]:
claw = solve(order=2, limiter=0)
make_plots(claw)

## Lax-Wendroff with minmod limiter

In [ ]:
claw = solve(order=2, limiter=pyclaw.limiters.tvd.minmod)
make_plots(claw)

## Lax-Wendroff with superbee limiter

In [ ]:
claw = solve(order=2, limiter=pyclaw.limiters.tvd.superbee)
make_plots(claw)

## Lax-Wendroff with MC limiter

In [ ]:
claw = solve(order=2, limiter=pyclaw.limiters.tvd.MC)
make_plots(claw)

## SharpClaw: 5'th order WENO

In [ ]:
claw = solve(solver_type='sharpclaw')
make_plots(claw)
del claw

## SharpClaw: 7'th order WENO

In [ ]:
claw = solve(solver_type='sharpclaw', weno_order=7)
make_plots(claw)
del claw

## SharpClaw: MC limiter

In [ ]:
claw = solve(solver_type='sharpclaw', lim_type=1, limiter=pyclaw.limiters.tvd.MC)
make_plots(claw)
del claw

## Animate solution

To examine the evolution more thoroughly, it's nice to see all the frames in sequence.  We can do this as follows.

In [ ]:
from clawpack.visclaw import ianimate
claw = solve(order=2, limiter=pyclaw.limiters.tvd.MC)
ianimate.ianimate(claw)